In [ ]:
import pathlib as pl
import numpy as np
import matplotlib.pyplot as plt
import swiapi
import flopy

# path to mf6 shared library
# executables based on mf6 feat-swi branch at: 
#   https://github.com/langevin-usgs/modflow6/tree/feat-swi
mf6exe = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/mf6"
libmf6 = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/libmf6.dylib"

In [ ]:
def add_model(sim, name):
    gwf = flopy.mf6.ModflowGwf(
        sim, 
        modelname=name, 
        save_flows=True,
        newtonoptions="newton",
    )
    dis = flopy.mf6.ModflowGwfdis(
        gwf, 
        nlay=nlay,
        nrow=nrow,
        ncol=ncol,
        delr=delr,
        delc=delc,
        top=top,
        botm=botm,
    )
    ic = flopy.mf6.ModflowGwfic(gwf, strt=0)
    npf = flopy.mf6.ModflowGwfnpf(
        gwf,
        save_specific_discharge=True,
        alternative_cell_averaging="harmonic",
        icelltype=icelltype,
        k=k,
    )
    sto = flopy.mf6.ModflowGwfsto(
        gwf,
        iconvert=1,
        ss=1.e-5, 
        sy=0.2
    )
    if name == "freshmodel":
        swi = flopy.mf6.ModflowGwfswi(gwf, zeta=-200.)
    else:
        swi = flopy.mf6.ModflowGwfswi(gwf, saltwater=True, zeta=-200.)

    cghb = 1. * delr * delc / 10.
    ghb = flopy.mf6.ModflowGwfghb(gwf, stress_period_data=[[0, 0, 0, h0, cghb],
                                                        [0, 0, ncol - 1, h1, cghb]])
    if name == "freshmodel":
        rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=recharge)
    budget_file = name + '.bud'
    head_file = name + '.hds'
    oc = flopy.mf6.ModflowGwfoc(
            gwf,
            budget_filerecord=budget_file,
            head_filerecord=head_file,
            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
            printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
    )
    return

In [ ]:
#create simple test model
Lx = 10000 # meters
delr, delc = 100., 1.
ncol = int(Lx / delr)
nlay = 1
nrow = 1
top = 50.
botm = -400.
recharge = 0.001
k = 10.
h0 = 0.
h1 = h0
icelltype = 1

ws = './two_model'
name = 'two_model'
sim = flopy.mf6.MFSimulation(
    sim_name=name, 
    sim_ws=ws, 
    exe_name=mf6exe,
    memory_print_option="all"
)
nper = 1
# nstp = 100
# perlen = 200000.
nstp = 1
perlen = 200000. / 100
perioddata = nper * [(perlen, nstp, 1.)]
tdis = flopy.mf6.ModflowTdis(sim, perioddata=perioddata)
ims = flopy.mf6.ModflowIms(
    sim,
    print_option="all",
    linear_acceleration="bicgstab",
)
fresh_model_name = "freshmodel"
salt_model_name = "saltmodel"
add_model(sim, fresh_model_name)
add_model(sim, salt_model_name)
sim.write_simulation()

In [ ]:
mf6_swi = swiapi.SwiAPI(libmf6, ws, fresh_model_name, salt_model_name)
mf6_swi.run(maxiter=30, verbose=True)
print("done...")